In [58]:
import scanpy as sc
import pandas as pd
import os

In [59]:
#s
adata = sc.read("/mnt/e/surfdrivesync/radboud/data/python/jupyter_notebooks/cma_meta_atlas.h5ad")
adata

AnnData object with n_obs × n_vars = 65036 × 27656
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'donors'
    var: 'features'

In [3]:
# Selecting the conditions where the ML algorithm was well calibrated
cond_list=["LSC","LESC","CE","Cj","qSK","SK","CF"]

def join_shap(c=str):
    shap=pd.read_csv(f'SHAP_cpredictor_032/{c}_shap_150.csv',sep='\t',index_col=0)
    shap["cell_state"] = c
    return shap

plot_list = [join_shap(c) for c in cond_list]
shap_full = pd.concat(plot_list, axis=0)

shap_unique = shap_full.index.unique().to_list()
print(len(shap_unique))

807


In [4]:
# Selecting the conditions where the ML algorithm was well calibrated
cond_list_hvg=["LSC","LESC","LE","CE","Cj","qSK","SK","CF","TSK","EC","IC","Mel","Ves","nm-cSC","MC"]
date_output = "18042024"

def join_hvg(c=str):
    hvg_150=pd.read_csv(f'HVG_meta_atlas/{date_output}/HVG_{c}_leiden_anno.tsv',sep='\t',index_col=0)
    hvg_150["cell_state"] = c
    hvg_150= hvg_150.head(150)
    return hvg_150

plot_list = [join_hvg(c) for c in cond_list_hvg]
hvg_full = pd.concat(plot_list, axis=0)

hvg_unique = hvg_full.index.unique().to_list()
print(len(hvg_unique))

1047


In [5]:
hybrid_features = shap_unique+hvg_unique
hybrid_features = list(set(hybrid_features))
print(len(hybrid_features))

1574


In [6]:
# Remove technical genes/features (e.g. MALAT1 & chromosome linked genes like XIST and SRY)
hybrid_features_sel = [i for i in hybrid_features if i not in ["MALAT1","XIST","SRY"]]
print(len(hybrid_features_sel))

1572


In [8]:
adata = adata[:, hybrid_features_sel]
adata.X

<65036x1572 sparse matrix of type '<class 'numpy.float64'>'
	with 39360438 stored elements in Compressed Sparse Row format>

In [9]:
missing_features = [i for i in ['PAX6', 'KRT14', 'S100A2', 'TP63', 'KRT15', 'GPHA2', 'CPVL', 'CXCL17', 'MUC1',
                                 'KRT7', 'S100A8', 'KRT3', 'KRT12', 'KRT24', 'AREG', 'VIM', 'LUM', 'KERA', 'CD34', 
                                 'MMP1', 'MMP2', 'MMP3', 'THY1', 'NT5E', 'FBLN1', 'COL1A1', 'PDGFRA', 'COL8A2', 'CA3', 
                                 'SLC4A11', 'ACKR1', 'PECAM1', 'LYVE1', 'MITF', 'CCL3', 'SOX10', 'CDH19', 'NGFR', 'SCN7A', 
                                 'ACTA2', 'NOTCH3'] if i not in hybrid_features_sel]
missing_features

# Only KRT7, NGFR and LYVE1 not in the list

['KRT7', 'LYVE1', 'NGFR']

In [10]:
adata.write_h5ad("SHAP_cpredictor_032/cma_meta_atlas.h5ad")

# Recursive feature elimination round 2

In [4]:
# Selecting the conditions where the ML algorithm was better calibrated after rfe
cond_list=["CE", "CF", "Cj", "EC", "LE", "LESC", "LSC", "Mel", "SK", "nm-CSC", "qSK"] # No IC, MC, TSK, Ves
def join_shap(c=str):
    shap=pd.read_csv(f'models_after_training_rfe2/{c}_shap_100.csv',sep='\t',index_col=0)
    shap["cell_state"] = c
    return shap

plot_list = [join_shap(c) for c in cond_list]
shap_full = pd.concat(plot_list, axis=0)

shap_unique = shap_full.index.unique().to_list()
print(len(shap_unique))

643


# rfe2 shap only

In [ ]:
# Selecting the conditions where the ML algorithm was better calibrated
cond_list=["CE", "CF", "Cj", "EC", "IC", "LE", "LESC", "LSC", "MC", "Mel", "SK", "TSK", "Ves", "nm-CSC", "qSK"]
def join_shap(c=str):
    shap=pd.read_csv(f'models_after_training_rfe2/{c}_shap_100.csv',sep='\t',index_col=0)
    shap["cell_state"] = c
    return shap

plot_list = [join_shap(c) for c in cond_list]
shap_full = pd.concat(plot_list, axis=0)

shap_unique = shap_full.index.unique().to_list()
print(len(shap_unique))

In [5]:
# Remove technical genes/features (e.g. MALAT1 & chromosome linked genes like XIST and SRY)
hybrid_features_sel = [i for i in shap_unique if i not in ["MALAT1", "XIST", "SRY", "MT1Z", "GAS5", "BTG2", "CD9", "SNHG32"]]
print(len(hybrid_features_sel))

836


In [6]:
adata = adata[:, hybrid_features_sel]
adata.X

<65036x836 sparse matrix of type '<class 'numpy.float64'>'
	with 23102660 stored elements in Compressed Sparse Row format>

In [7]:
missing_features = [i for i in ['PAX6', 'KRT14', 'S100A2', 'TP63', 'KRT15', 'GPHA2', 'CPVL', 'CXCL17', 'MUC1',
                                 'KRT7', 'S100A8', 'KRT3', 'KRT12', 'KRT24', 'AREG', 'VIM', 'LUM', 'KERA', 'CD34', 
                                 'MMP1', 'MMP2', 'MMP3', 'THY1', 'NT5E', 'FBLN1', 'COL1A1', 'PDGFRA', 'COL8A2', 'CA3', 
                                 'SLC4A11', 'ACKR1', 'PECAM1', 'LYVE1', 'MITF', 'CCL3', 'SOX10', 'CDH19', 'NGFR', 'SCN7A', 
                                 'ACTA2', 'NOTCH3'] if i not in hybrid_features_sel]
missing_features

# missing_some check which classes are calibrated best without top 100 hvgs

['CXCL17',
 'MUC1',
 'KRT7',
 'S100A8',
 'MMP3',
 'THY1',
 'NT5E',
 'COL8A2',
 'CA3',
 'LYVE1',
 'ACTA2']

In [8]:
adata.write_h5ad("SHAP_cpredictor_032_100hvg/shap_only/cma_meta_atlas.h5ad")

# rfe2 shap joined with top 100 hvgs (eventually used for the final model)

In [5]:
# Selecting all conditions
cond_list_hvg=["LSC","LESC","LE","CE","Cj","qSK","SK","CF","TSK","EC","IC","Mel","Ves","nm-cSC","MC"]
date_output = "18042024"

def join_hvg(c=str):
    hvg=pd.read_csv(f'HVG_meta_atlas/{date_output}/HVG_{c}_leiden_anno.tsv',sep='\t',index_col=0)
    hvg["cell_state"] = c
    hvg= hvg.head(100)
    return hvg

plot_list = [join_hvg(c) for c in cond_list_hvg]
hvg_full = pd.concat(plot_list, axis=0)

hvg_unique = hvg_full.index.unique().to_list()
print(len(hvg_unique))

1047


In [6]:
hybrid_features = shap_unique+hvg_unique
hybrid_features = list(set(hybrid_features))
print(len(hybrid_features))

1249


In [7]:
# Remove technical genes/features (e.g. MALAT1 & chromosome linked genes like XIST and SRY)
hybrid_features_sel = [i for i in hybrid_features if i not in ["MALAT1", "XIST", "SRY", "MT1Z", "GAS5", "BTG2", "CD9", "SNHG32"]]
print(len(hybrid_features_sel))

1243


In [8]:
adata = adata[:, hybrid_features_sel]
adata.X

<65036x1243 sparse matrix of type '<class 'numpy.float64'>'
	with 33048022 stored elements in Compressed Sparse Row format>

In [10]:
missing_features = [i for i in ['PAX6', 'KRT14', 'S100A2', 'TP63', 'KRT15', 'GPHA2', 'CPVL', 'CXCL17', 'MUC1',
                                 'KRT7', 'S100A8', 'KRT3', 'KRT12', 'KRT24', 'AREG', 'VIM', 'LUM', 'KERA', 'CD34', 
                                 'MMP1', 'MMP2', 'MMP3', 'THY1', 'NT5E', 'FBLN1', 'COL1A1', 'PDGFRA', 'COL8A2', 'CA3', 
                                 'SLC4A11', 'ACKR1', 'PECAM1', 'LYVE1', 'MITF', 'CCL3', 'SOX10', 'CDH19', 'NGFR', 'SCN7A', 
                                 'ACTA2', 'NOTCH3'] if i not in hybrid_features_sel]
missing_features

# Only KRT7 and LYVE1 not in the list

['KRT7', 'LYVE1']

In [11]:
adata.write_h5ad("SHAP_cpredictor_032_100hvg/cma_meta_atlas.h5ad")

# rfe3 shap only (below not used due to missing important features)

In [3]:
# Selecting the conditions where the ML algorithm was better calibrated
cond_list=["CE", "CF", "Cj", "EC", "IC", "LE", "LESC", "LSC", "MC", "Mel", "SK", "TSK", "Ves", "nm-CSC", "qSK"]
def join_shap(c=str):
    shap=pd.read_csv(f'models_after_training_rfe3/{c}_shap_50.csv',sep='\t',index_col=0)
    shap["cell_state"] = c
    return shap

plot_list = [join_shap(c) for c in cond_list]
shap_full = pd.concat(plot_list, axis=0)

shap_unique = shap_full.index.unique().to_list()
print(len(shap_unique))

452


In [4]:
# Remove technical genes/features (e.g. MALAT1 & chromosome linked genes like XIST and SRY)
hybrid_features_sel = [i for i in shap_unique if i not in ["MALAT1", "XIST", "SRY", "MT1Z", "GAS5", "BTG2", "CD9", "SNHG32", "OLFM1", "RPL13", "RPL18A", "PFDN5", "SAT1", "MIF", "IFI16"]]
print(len(hybrid_features_sel))

445


In [5]:
adata = adata[:, hybrid_features_sel]
adata.X

<65036x445 sparse matrix of type '<class 'numpy.float64'>'
	with 13075249 stored elements in Compressed Sparse Row format>

In [6]:
missing_features = [i for i in ['PAX6', 'KRT14', 'S100A2', 'TP63', 'KRT15', 'GPHA2', 'CPVL', 'CXCL17', 'MUC1',
                                 'KRT7', 'S100A8', 'KRT3', 'KRT12', 'KRT24', 'AREG', 'VIM', 'LUM', 'KERA', 'CD34', 
                                 'MMP1', 'MMP2', 'MMP3', 'THY1', 'NT5E', 'FBLN1', 'COL1A1', 'PDGFRA', 'COL8A2', 'CA3', 
                                 'SLC4A11', 'ACKR1', 'PECAM1', 'LYVE1', 'MITF', 'CCL3', 'SOX10', 'CDH19', 'NGFR', 'SCN7A', 
                                 'ACTA2', 'NOTCH3'] if i not in hybrid_features_sel]
missing_features

# missing_some check which classes are calibrated best without top 150 & 100 hvgs

['TP63',
 'CXCL17',
 'MUC1',
 'KRT7',
 'S100A8',
 'KRT12',
 'AREG',
 'CD34',
 'MMP2',
 'MMP3',
 'THY1',
 'NT5E',
 'PDGFRA',
 'CA3',
 'LYVE1',
 'MITF',
 'NGFR',
 'ACTA2',
 'NOTCH3']

In [7]:
adata.write_h5ad("SHAP_cpredictor_032_50hvg/shap_only/cma_meta_atlas.h5ad")

# rfe3 shap joined with top 50 hvgs

In [21]:
# Selecting all conditions
cond_list_hvg=["LSC","LESC","LE","CE","Cj","qSK","SK","CF","TSK","EC","IC","Mel","Ves","nm-cSC","MC"]
date_output = "18042024"

def join_hvg(c=str):
    hvg=pd.read_csv(f'HVG_meta_atlas/{date_output}/HVG_{c}_leiden_anno.tsv',sep='\t',index_col=0)
    hvg["cell_state"] = c
    hvg= hvg.head(50)
    return hvg

plot_list = [join_hvg(c) for c in cond_list_hvg]
hvg_full = pd.concat(plot_list, axis=0)

hvg_unique = hvg_full.index.unique().to_list()
print(len(hvg_unique))

583


In [22]:
hybrid_features = shap_unique+hvg_unique
hybrid_features = list(set(hybrid_features))
print(len(hybrid_features))

752


In [23]:
# Remove technical genes/features (e.g. MALAT1 & chromosome linked genes like XIST and SRY)
hybrid_features_sel = [i for i in hybrid_features if i not in ["MALAT1", "XIST", "SRY", "MT1Z", "GAS5", "BTG2", "CD9", "SNHG32", "OLFM1", "RPL13", "RPL18A", "PFDN5", "SAT1", "MIF", "IFI16"]]
print(len(hybrid_features_sel))

740


In [24]:
adata = adata[:, hybrid_features_sel]
adata.X

<65036x740 sparse matrix of type '<class 'numpy.float64'>'
	with 21260660 stored elements in Compressed Sparse Row format>

In [25]:
missing_features = [i for i in ['PAX6', 'KRT14', 'S100A2', 'TP63', 'KRT15', 'GPHA2', 'CPVL', 'CXCL17', 'MUC1',
                                 'KRT7', 'S100A8', 'KRT3', 'KRT12', 'KRT24', 'AREG', 'VIM', 'LUM', 'KERA', 'CD34', 
                                 'MMP1', 'MMP2', 'MMP3', 'THY1', 'NT5E', 'FBLN1', 'COL1A1', 'PDGFRA', 'COL8A2', 'CA3', 
                                 'SLC4A11', 'ACKR1', 'PECAM1', 'LYVE1', 'MITF', 'CCL3', 'SOX10', 'CDH19', 'NGFR', 'SCN7A', 
                                 'ACTA2', 'NOTCH3'] if i not in hybrid_features_sel]
missing_features

# Only KRT7 and LYVE1 not in the list

['TP63', 'MUC1', 'KRT7', 'CD34', 'THY1', 'NT5E', 'LYVE1', 'NGFR']

In [26]:
adata.write_h5ad("SHAP_cpredictor_032_50hvg/cma_meta_atlas.h5ad")

# rfe3 after calibration check

In [81]:
# Selecting the conditions where the ML algorithm was better calibrated
cond_list=["CE", "CF", "Cj", "EC", "IC", "LE", "LSC","MC", "SK", "TSK", "Ves", "nm-CSC", "qSK"] # No Mel and LESC
def join_shap(c=str):
    shap=pd.read_csv(f'models_after_training_rfe3/{c}_shap_50.csv',sep='\t',index_col=0)
    shap["cell_state"] = c
    return shap

plot_list = [join_shap(c) for c in cond_list]
shap_full = pd.concat(plot_list, axis=0)

shap_unique = shap_full.index.unique().to_list()
print(len(shap_unique))

417


In [82]:
# Selecting the conditions where the ML algorithm was better calibrated
cond_list=["Mel","LESC"]
def join_shap(c=str):
    shap=pd.read_csv(f'models_after_training_rfe2/{c}_shap_100.csv',sep='\t',index_col=0)
    shap["cell_state"] = c
    return shap

plot_list = [join_shap(c) for c in cond_list]
shap_full = pd.concat(plot_list, axis=0)

shap_unique_mellesc = shap_full.index.unique().to_list()
print(len(shap_unique_mellesc))

192


In [83]:
# Selecting all conditions
cond_list_hvg=["LE","qSK","EC","IC","Ves","nm-cSC","MC","Mel"] # Only no TSK and SK
date_output = "18042024"

def join_hvg(c=str):
    hvg=pd.read_csv(f'HVG_meta_atlas/{date_output}/HVG_{c}_leiden_anno.tsv',sep='\t',index_col=0)
    hvg["cell_state"] = c
    hvg= hvg.head(50)
    return hvg

plot_list = [join_hvg(c) for c in cond_list_hvg]
hvg_full = pd.concat(plot_list, axis=0)

hvg_unique = hvg_full.index.unique().to_list()
print(len(hvg_unique))

356


In [84]:
hybrid_features = shap_unique+hvg_unique+shap_unique_mellesc
hybrid_features = list(set(hybrid_features))
print(len(hybrid_features))

687


In [85]:
# Remove technical genes/features (e.g. MALAT1 & chromosome linked genes like XIST and SRY)
hybrid_features_sel = [i for i in hybrid_features if i not in ["MALAT1", "XIST", "SRY", "MT1Z", "GAS5", "BTG2", "CD9", "SNHG32", "OLFM1", "RPL13", "RPL18A", "PFDN5", "SAT1", "MIF", "IFI16"]]
print(len(hybrid_features_sel))

677


In [86]:
missing_features = [i for i in ['PAX6', 'KRT14', 'S100A2', 'TP63', 'KRT15', 'GPHA2', 'CPVL', 'CXCL17', 'MUC1',
                                 'KRT7', 'S100A8', 'KRT3', 'KRT12', 'KRT24', 'AREG', 'VIM', 'LUM', 'KERA', 'CD34', 
                                 'MMP1', 'MMP2', 'MMP3', 'THY1', 'NT5E', 'FBLN1', 'COL1A1', 'PDGFRA', 'COL8A2', 'CA3', 
                                 'SLC4A11', 'ACKR1', 'PECAM1', 'LYVE1', 'MITF', 'CCL3', 'SOX10', 'CDH19', 'NGFR', 'SCN7A', 
                                 'ACTA2', 'NOTCH3'] if i not in hybrid_features_sel]
missing_features

# Only KRT7 and LYVE1 not in the list

['TP63',
 'CXCL17',
 'MUC1',
 'KRT7',
 'S100A8',
 'CD34',
 'MMP2',
 'MMP3',
 'THY1',
 'NT5E',
 'PDGFRA',
 'LYVE1',
 'NGFR']

In [87]:
adata = adata[:, hybrid_features_sel]
adata.X

<65036x677 sparse matrix of type '<class 'numpy.float64'>'
	with 18888020 stored elements in Compressed Sparse Row format>

In [88]:
adata.write_h5ad("SHAP_cpredictor_032_50hvg/selected/cma_meta_atlas.h5ad")

# Optimal hyperparameters: used object from rfe2 shap joined with top 100 hvgs

# OLD

In [24]:
# Old to test
%%timeit
cond_list=["LSC","LESC","CE","Cj","qSK","SK","CF"]

plot_list=[]
for value in cond_list:    
    c = value
    results = [each for each in os.listdir(f'SHAP_cpredictor_032/') if each.endswith('shap_150.csv')]
    shap = results[0]
    shap=pd.read_csv(f'SHAP_cpredictor_032/{c}_shap_150.csv',sep='\t',index_col=0)
    shap["cell_state"] = c
    plot_list.append(shap)
shap_full = pd.concat(plot_list, axis=0)
shap_full

UsageError: Line magic function `%%timeit` not found.


In [27]:
# Selecting the conditions where the ML algorithm was not well calibrated
cond_list=["LSC","LESC","CE","Cj","qSK","SK","CF"]

def join_hvg(c=str):
    results = [each for each in os.listdir(f'SHAP_cpredictor_032/') if each.endswith('shap_150.csv')]
    shap = results[0]
    shap=pd.read_csv(f'SHAP_cpredictor_032/{c}_shap_150.csv',sep='\t',index_col=0)
    shap["cell_state"] = c
    return shap

plot_list = [join_shap(value=c) for c in cond_list]
shap_full = pd.concat(plot_list, axis=0)
shap_full

,feature_importance_vals,cell_state
col_name,,
GPHA2,0.004786,LSC
CLDN4,0.004634,LSC
MGST1,0.002822,LSC
MT1X,0.002682,LSC
AQP3,0.002095,LSC
...,...,...
ANXA5,0.000303,CF
AC083843.3,0.000299,CF
TMEM219,0.000298,CF
